<a href="https://colab.research.google.com/github/Hashhhhhhhh/JAX-Playground/blob/main/Genetic_Algorithm_step_Crossover_and_mutation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perform 1 GA iteration: selects states based on fidelity, apply crossover (Ex: convex combination of state vectors), and mutate. This simulates evolving quantum states toward a metrology-optimal state.

In [1]:
!pip install jax

In [19]:
import jax.numpy as jnp
from jax import jit, vmap,random
#paramters
N=2
pop_size=5
target=jnp.array([1.0,0.0,0.0,1.0])/jnp.sqrt(2)
key=random.PRNGKey(14)
#initializing states
key,subkey=random.split(key)
states=random.normal(subkey,(2**N,pop_size))+1j*random.normal(subkey,(2**N,pop_size))
states=states/jnp.linalg.norm(states)
print("States.", "Each column corresponds to one state vector.")
print(states)
#fitness ---> fidelity
fidelity = vmap(lambda state: jnp.abs(jnp.vdot(state, target))**2, in_axes=1)(states)
print("\nFidelity")
print(fidelity)
#selection : top2 states
key,subkey=random.split(key)
idx=jnp.argsort(fidelity)[::-1][2:]
#[::-1] reverses the sorted indices to get descending order (highest fidelity first)
#Top 2 fidelities
parents=states[:,idx]
print()
print("Parents")
print(parents)
#Crossover
alpha=0.5
child1=alpha*parents[:,0]+(1-alpha)*parents[:,1]
child1=child1/jnp.linalg.norm(child1)
child2=(1-alpha)*parents[:,0]+(alpha)*parents[:,1]
child2=child2/jnp.linalg.norm(child2)
child=jnp.stack((child1,child2))
print()
print("Children")
print(child)
#mutation
key,subkey=random.split(key)
noise=0.1*(random.normal(subkey,(2**N,))+1j*random.normal(subkey,(2**N,)))
child1=child1+noise
child1=child1/jnp.linalg.norm(child1)
new_pop=jnp.column_stack([parents,child1,child2])
print()
print("New population Fidelities")
print(vmap(lambda state: jnp.abs(jnp.vdot(state, target))**2, in_axes=1)(new_pop))

States. Each column corresponds to one state vector.
[[ 0.0520511 +0.0520511j   0.06296879+0.06296879j -0.29019168-0.29019168j
   0.21126571+0.21126571j  0.31737888+0.31737888j]
 [ 0.07345413+0.07345413j  0.14589515+0.14589515j  0.04740075+0.04740075j
  -0.05666243-0.05666243j  0.01700507+0.01700507j]
 [-0.17689225-0.17689225j -0.09186433-0.09186433j  0.14294404+0.14294404j
   0.09566636+0.09566636j  0.04103819+0.04103819j]
 [-0.01764419-0.01764419j -0.17997827-0.17997827j -0.00977259-0.00977259j
   0.14289558+0.14289558j -0.32727218-0.32727218j]]

Fidelity
[1.1838353e-03 1.3691216e-02 8.9978553e-02 1.2543024e-01 9.7877346e-05]

Parents
[[ 0.06296879+0.06296879j  0.0520511 +0.0520511j   0.31737888+0.31737888j]
 [ 0.14589515+0.14589515j  0.07345413+0.07345413j  0.01700507+0.01700507j]
 [-0.09186433-0.09186433j -0.17689225-0.17689225j  0.04103819+0.04103819j]
 [-0.17997827-0.17997827j -0.01764419-0.01764419j -0.32727218-0.32727218j]]

Children
[[ 0.19575003+0.19575003j  0.37330613+0.3733